In [1]:
import os
from datetime import datetime, timedelta
import torch
import numpy as np
import h5py
from episode import Episode




Radar Episode Manager


In [4]:
import os
import torch
from episode import Episode

class RadarEpisodeManager:
    def __init__(self, year, path, episode_dir):
        self.path = path
        self.episode_dir = episode_dir
        self.year=year

    def save_radar_data(self, year, save_frequency: str) -> None:
        current_week = 0
        for month in range(1, 13):
            start_date = datetime(year, month, 1)
            end_date = start_date.replace(day=1, month=month + 1) if month < 12 else start_date.replace(year=self.year + 1, month=1)
            num_days = (end_date - start_date).days
            
            output = []
            weekly_output = []
            daily_output = []
            
        
        
        # Iterate over all days in the current month
            for day in range(1, num_days + 1):
                # Iterate over hours from 00:00 to 23:55
                for hour in range(0, 24):
                    # Iterate over minutes from 00:00 to 23:55 in 5-minute intervals
                    for minute in range(0, 60, 5):
                        # Construct the file path
                        file_name = f'RAD_NL25_RAP_5min_{self.year}{month:02d}{day:02d}{hour:02d}{minute:02d}.h5'
                        root_direction = os.path.join(self.path, str(self.year), f'{month:02d}', file_name)
                        
                        # Load and process the image data
                        image = np.array(h5py.File(root_direction)['image1']['image_data'])
                        image = image[264:520, 242:498]
                        image[image == 65535] = 0
                        image = image.astype('float32')
                        image = image / 100 * 12
                        image = np.clip(image, 0, 128)
                        image = image / 40
                        output.append(image)
                
                
                
                if save_frequency == "daily":
                    daily_output=output
                    episode_dir = os.path.join(self.episode_dir)
                    os.makedirs(episode_dir, exist_ok=True)  # Create the episode directory if it doesn't exist
                    
                    daily_output = np.array(daily_output)
                    observations = torch.tensor(daily_output)
                    actions = torch.zeros(observations.shape[0], dtype=torch.long)
                    rewards = torch.zeros(observations.shape[0], dtype=torch.float)
                    ends = torch.zeros(observations.shape[0], dtype=torch.long)
                    mask_padding = torch.zeros(observations.shape[0], dtype=torch.bool)

                    episode = Episode(observations, actions, rewards, ends, mask_padding)
                    episode_id = f"{day:02d}"
                    episode.save(os.path.join(episode_dir, f'episode_day_{self.year}_{month:02d}_{episode_id}.pt'))
                    output = []
                    daily_output=[]
                
                    
                if day % 7 == 0:
                    weekly_output.append(output)
                    current_week += 1
                    if save_frequency == "weekly":
                        episode_dir = os.path.join(self.episode_dir)
                        os.makedirs(episode_dir, exist_ok=True)  # Create the episode directory if it doesn't exist
                        
                        weekly_output = np.array(weekly_output)
                        weekly_output = np.concatenate(weekly_output, axis=0)
                        observations = torch.tensor(weekly_output)
                        print(observations.size())
                        actions = torch.zeros(observations.shape[0], dtype=torch.long)
                        print(actions.size())
                        rewards = torch.zeros(observations.shape[0], dtype=torch.float)
                        ends = torch.zeros(observations.shape[0], dtype=torch.long)
                        mask_padding = torch.zeros(observations.shape[0], dtype=torch.bool)

                        episode = Episode(observations, actions, rewards, ends, mask_padding)
                        episode_id = f"{current_week}"
                        episode.save(os.path.join(episode_dir, f'episode_week_{self.year}_{month:02d}_{episode_id}.pt'))
                        weekly_output = []
                        output = []

            if save_frequency == "monthly":
                episode_dir = os.path.join(self.episode_dir)
                os.makedirs(episode_dir, exist_ok=True)  # Create the episode directory if it doesn't exist
                output = np.array(output)
                observations = torch.tensor(output)
                actions = torch.zeros(observations.shape[0], dtype=torch.long)
                rewards = torch.zeros(observations.shape[0], dtype=torch.float)
                ends = torch.zeros(observations.shape[0], dtype=torch.long)
                mask_padding = torch.zeros(observations.shape[0], dtype=torch.bool)
                episode = Episode(observations, actions, rewards, ends, mask_padding)
                episode_id = f"{month:02d}"
                episode.save(os.path.join(episode_dir, f'episode_month_{self.year}_{episode_id}.pt'))
                output=[]
                


In [5]:
from episode import Episode
from utils import EpisodeDirManager


In [7]:
import torch

# Load the .pt file
episode_path = '/space/zboucher/training_dataset/episode_month_2008_01.pt'
path='/home/hbi/RAD_NL25_RAP_5min'
episode = torch.load(episode_path)

# Access the data from the episode
observations = episode['observations']
actions = episode['actions']
rewards = episode['rewards']
ends = episode['ends']
mask_padding = episode['mask_padding']

# Process the data as needed
# ...

# Print the shape of the observations tensor
print(observations.shape)




torch.Size([8928, 256, 256])
